## Connecting Google Drive to this notebook to match any external files we have created and utilize in this part.

In [ ]:
# link this notebook to my google drive so I can retrieve data files from there
# Import the dedicated library
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Importing libraries and dependencies

In [ ]:
%pip install spacy_transformers  # library needed to (re)train spacy ner models

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.8/197.8 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 27.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manyli

In [ ]:
# data loading & handling
import numpy as np
import pandas as pd



# NER
import nltk
from nltk import ne_chunk, pos_tag, word_tokenize, sent_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import wordnet
# for stopwords, I will only drop those of small length that could mess with abbreviations
stop_words = [word for word in stopwords.words('english') if len(word) <= 3]
import spacy
from spacy.tokens import DocBin
from spacy import displacy
# There are server blocking issues that might arise when trying to call
# displacy.serve multiple times through a for loop, so we save the html of the
# corresponding results and print that HTML snippet on every step
from IPython.display import display, HTML



# Miscellaneous utilities
from tqdm import tqdm
import json
import os
import re

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
import spacy_transformers  # essential in order to call transformer-based models

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [ ]:
!pip install google-cloud-bigquery
!pip install db-dtypes


In [ ]:
from google.colab import auth
from google.cloud import bigquery
auth.authenticate_user()

# BigQuery client
project_id = 'nlp-project-427710'
client = bigquery.Client(project=project_id)

# SQL query to fetch data from BigQuery table
query = """
    SELECT status_id, text
    FROM `nlp-project-427710.covid.uns_covid1`
    WHERE lang='en'
    LIMIT 100000
"""

# Fetch data me to BigQuery client
df = client.query(query).to_dataframe()

In [ ]:
df.head()

,status_id,text
0,1245406137236615169,COVID-19 India Update : Total Active Cases : 1...
1,1245415621648678912,U.S.: #Florida governor issues stay-at-home or...
2,1245425967469924354,Guyana: Aviation authority extends suspension ...
3,1245455807812173824,"Canada: #WindsorInternationalAirport, #Ontario..."
4,1245492693368819715,"Canada: #AirCanada suspends flights to, from s..."


In [ ]:
# loading our best performing model
ner1 = spacy.load('/content/drive/MyDrive/NLP_project/sample_custom_ner/config/output/config_en_gpu_accuracy_null.cfg_train_size_500/model-last')

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [ ]:
# Testing it with a sample text
doc = ner1("300000 deaths from covid19 and lung disease")
for ent in doc.ents:
    print(ent.text, ent.label_)

covid19 GENERIC


In [ ]:
from IPython.display import display, HTML


In [ ]:
for text in df['text']:
  doc = ner1(text)
  label_types = set([ent.label_ for ent in doc.ents])
  if (len(label_types) >= 2):
    html = displacy.render(doc, style="ent")
    display(HTML(html))
    print('\n\n')
  else:
    pass



<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

...

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

KeyboardInterrupt: 

In [ ]:
from spacy import displacy
# There are server blocking issues that might arise when trying to call
# displacy.serve multiple times through a for loop, so we save the html of the
# corresponding results and print that HTML snippet on every step
from IPython.display import display, HTML
from spacy.tokens import Doc, Span


In [ ]:
# using 'displacy' on 2 samples to compare real vs predicted annotations from the optimized Maximum Entropy model
os.chdir('/content/drive/MyDrive/NLP_project/sample_custom_ner/')
# files to use to showcase our examples
ex1_files = ['example1_real.json', 'example1_pred.json']
ex2_files = ['example2_real.json', 'example2_pred.json']

# extracting json details to render using displacy
def render_annotations(text, entities, title):
    displacy_data = {
        "text": text,
        "ents": [{"start": start, "end": end, "label": label} for start, end, label in entities],
        "title": title
    }
    return displacy.render(displacy_data, style="ent", options=options, manual=True)

In [ ]:
titles_to_use = ["Example 1 - Real", "Example 1 - Pred", "Example 2 - Real", "Example 2 - Pred"]

# loading a blank spacy model to be able to use displacy properly
nlp = spacy.blank("en")

# specifying the labels to show based on those evident in the specific samples to show
options = {
    "ents": ["B-ORG", "B-GENERIC", "I-GENERIC"]
    }


def json_load_to_html(files, titles):
    for i, file_path in enumerate(files):
        with open(file_path, 'r') as file:
            ex = json.load(file)
        for annotation in ex['annotations']:
            text = annotation[0]
            entities = annotation[1]['entities']
            html = render_annotations(text, entities, titles[i])
            display(HTML(html))

# Process and render annotations for example 1
json_load_to_html(ex1_files, titles_to_use[:2])

# Process and render annotations for example 2
json_load_to_html(ex2_files, titles_to_use[2:])


<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

In [ ]:
### Same for PromptNER

# first I show its results per-class and overall in a tabular format
d = {'CONDITIONS': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 2}, 'ENERIC': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 113}, 'ETRICS': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 12}, 'OLICIES': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 10}, 'REATMENTS': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1}, 'YMPTOMS': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 5}, 'overall_precision': 0.0, 'overall_recall': 0.0, 'overall_f1': 0.0, 'overall_accuracy': 0.9024624240486089}
pd.DataFrame.from_dict(d)

,CONDITIONS,ENERIC,ETRICS,OLICIES,REATMENTS,YMPTOMS,overall_precision,overall_recall,overall_f1,overall_accuracy
precision,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.902462
recall,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.902462
f1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.902462
number,2.0,113.0,12.0,10.0,1.0,5.0,0.0,0.0,0.0,0.902462


from matplotlib import pyplot as plt
_df_0['CONDITIONS'].plot(kind='hist', bins=20, title='CONDITIONS')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['ENERIC'].plot(kind='hist', bins=20, title='ENERIC')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['ETRICS'].plot(kind='hist', bins=20, title='ETRICS')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_3['OLICIES'].plot(kind='hist', bins=20, title='OLICIES')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_18['index'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_18, x='ENERIC', y='index', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_19['index'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_19, x='ETRICS', y='index', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_20['index'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_20, x='OLICIES', y='index', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)